# Leader Capacity Dashboard - Data Engineering Notebook

## 📋 Table of Contents
1. [Cell 1-3] Project Overview & Setup
2. [Cell 4-10] Data Loading
3. [Cell 11-14] Data Processing
4. [Cell 15-16] Dashboard Structure
5. [Cell 17-18] Data Quality
6. [Cell 19-20] Export & Next Steps

## [Cell 1] Project Overview

This notebook handles the data engineering pipeline for recreating a leader capacity dashboard. The dashboard will display current month plus three future months of:
- Booked time as % of available working time
- Vacation/leave data
- Salesforce opportunity data with likelihood and dates

### Data Sources
All data files are located in the `../data/` directory:
1. **10k Data for S3 (1).csv** - Time booking/allocation data
2. **10k Users.csv** - User roles and demographics (filtered to leadership roles)
3. **Namely Vacation and Leave Dataset.csv** - Employee vacation and leave records
4. **Salesforce Opportunity Data.csv** - Sales opportunities with probability and schedule
5. **Working Hours For US.csv** - US working hours and holidays
6. **UAE Working Hours.csv** - UAE working hours and holidays

In [ ]:
# [Cell 2] Package Installation Check
# ✅ All packages have been installed in the virtual environment!

# If you ever need to reinstall packages:
# !pip install pandas numpy matplotlib seaborn openpyxl

print("✅ All required packages are installed!")
print("📍 Using virtual environment at: venv/")
print("🚀 You can proceed to Cell 3 to import the libraries")

In [1]:
# [Cell 3] Import Required Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import calendar
import warnings
warnings.filterwarnings('ignore')

# For visualization (optional)
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✅ Libraries imported successfully")

Matplotlib is building the font cache; this may take a moment.


✅ Libraries imported successfully


## [Cell 4] Data Loading Section

### 🔄 Load all data sources with error handling
We'll load each CSV file and explore its structure to understand what data we're working with.

In [6]:
# [Cell 5] Load and Process 10k Data - FILTERED BY SELECTED ROLES ONLY

# Define the roles we want to include
selected_roles = [
    'Design',
    'Principal', 
    'Program Management',
    'Strategy',
    'Studio',
    'Tech'
]

print("📋 Selected roles to filter:")
for role in selected_roles:
    print(f"  ✓ {role}")
print(f"\n📊 Total roles selected: {len(selected_roles)}")
print()

# Load 10k Data
try:
    df_10k_data = pd.read_csv('../data/10k Data for S3 (1).csv')
    print(f"✅ 10k Data loaded successfully: {df_10k_data.shape}")
    print(f"   Columns: {df_10k_data.columns.tolist()[:10]}...")  # Show first 10 columns
except Exception as e:
    print(f"❌ Error loading 10k data: {e}")
    df_10k_data = None

# Load 10k Users
try:
    df_10k_users = pd.read_csv('../data/10k Users.csv')  # Note: capital 'U' in Users
    print(f"\n✅ 10k Users loaded successfully: {df_10k_users.shape}")
    print(f"   Columns: {df_10k_users.columns.tolist()}")
    
    # Check what roles are in the data
    if 'role' in df_10k_users.columns:
        print(f"\n📊 Available roles in the data:")
        role_counts = df_10k_users['role'].value_counts()
        print(role_counts.head(20))
    elif 'Role' in df_10k_users.columns:
        print(f"\n📊 Available roles in the data:")
        role_counts = df_10k_users['Role'].value_counts()
        print(role_counts.head(20))
    else:
        print("\n⚠️  No 'role' column found. Available columns:")
        print(df_10k_users.columns.tolist())
        
except Exception as e:
    print(f"❌ Error loading 10k users: {e}")
    df_10k_users = None

# Filter users by selected roles
if df_10k_users is not None:
    # Find the correct role column name (could be 'role', 'Role', 'job_title', etc.)
    role_column = None
    for col in ['role', 'Role', 'job_title', 'Job_Title', 'position', 'Position']:
        if col in df_10k_users.columns:
            role_column = col
            break
    
    if role_column:
        print(f"\n🔍 Filtering by role column: '{role_column}'")
        
        # Filter users by selected roles
        df_filtered_users = df_10k_users[df_10k_users[role_column].isin(selected_roles)]
        print(f"✅ Filtered users: {df_filtered_users.shape[0]} out of {df_10k_users.shape[0]} total users")
        
        # Show role distribution after filtering
        print(f"\n📊 Role distribution after filtering:")
        print(df_filtered_users[role_column].value_counts())
    else:
        print("\n⚠️  Could not find role column. Please check column names.")
        df_filtered_users = df_10k_users
else:
    df_filtered_users = None

# Merge 10k data with filtered users
if df_10k_data is not None and df_filtered_users is not None:
    print("\n🔗 Merging datasets...")
    
    # Check for ID columns
    print(f"\n📋 ID columns check:")
    if 'user_id' in df_10k_data.columns:
        print(f"  ✓ Found 'user_id' in 10k data")
    if 'id' in df_filtered_users.columns:
        print(f"  ✓ Found 'id' in users data")
    
    # Perform the merge
    try:
        df_merged = pd.merge(
            df_10k_data,
            df_filtered_users,
            left_on='user_id',
            right_on='id',
            how='inner'  # Only keep records that match
        )
        print(f"\n✅ Merge successful! Result: {df_merged.shape}")
        print(f"   {df_merged.shape[0]} records for {df_merged['id'].nunique()} unique users")
        
        # Save to use in later cells
        df_10k = df_merged
        
    except Exception as e:
        print(f"\n❌ Error during merge: {e}")
        print("Please check that the ID columns exist and match correctly")
        df_10k = None
else:
    df_10k = None

# Display sample of merged data
if df_10k is not None:
    print("\n📊 Sample of merged data:")
    # Show a few key columns
    sample_cols = ['user_id', 'id', role_column if role_column else 'role'] + \
                  [col for col in df_10k.columns if 'hours' in col.lower() or 'date' in col.lower()][:3]
    sample_cols = [col for col in sample_cols if col in df_10k.columns]
    print(df_10k[sample_cols].head())
    
    # VERIFICATION: Confirm that ONLY selected roles are in the final dataset
    print("\n✅ VERIFICATION - Confirming filtered data contains ONLY selected roles:")
    
    # Check what roles are actually in the merged data
    if role_column and role_column in df_10k.columns:
        actual_roles = df_10k[role_column].unique()
        print(f"\n📋 Roles found in final dataset:")
        for role in sorted(actual_roles):
            if role in selected_roles:
                print(f"  ✓ {role} (expected)")
            else:
                print(f"  ❌ {role} (UNEXPECTED - should not be here!)")
        
        # Double-check: Are all roles in the data part of selected_roles?
        unexpected_roles = [r for r in actual_roles if r not in selected_roles]
        if unexpected_roles:
            print(f"\n⚠️  WARNING: Found {len(unexpected_roles)} unexpected roles in the data!")
            print(f"These roles should NOT be in the filtered data: {unexpected_roles}")
        else:
            print(f"\n✅ SUCCESS: All roles in the final dataset are from the selected list!")
            print(f"   - {len(actual_roles)} unique roles found")
            print(f"   - {df_10k.shape[0]} total records")
            print(f"   - {df_10k['id'].nunique()} unique users")
    
    # Show final role distribution
    print(f"\n📊 Final role distribution in filtered 10k data:")
    if role_column and role_column in df_10k.columns:
        print(df_10k[role_column].value_counts())

📋 Selected roles to filter:
  ✓ Design
  ✓ Principal
  ✓ Program Management
  ✓ Strategy
  ✓ Studio
  ✓ Tech

📊 Total roles selected: 6

✅ 10k Data loaded successfully: (173658, 19)
   Columns: ['role', 'discipline', 'client', 'phase_name', 'incurred_hours', 'scheduled_hours', 'difference_from_past_scheduled_hours', 'future_scheduled_hours', 'total_hours', 'RequestTodayDate']...

✅ 10k Users loaded successfully: (1045, 36)
   Columns: ['last_login_time', 'billrate', 'id', 'first_name', 'last_name', 'account_owner', 'archived', 'billability_target', 'billable', 'created_at', 'deleted', 'deleted_at', 'discipline', 'display_name', 'email', 'employee_number', 'guid', 'hire_date', 'invitation_pending', 'license_type', 'location', 'location_id', 'mobile_phone', 'office_phone', 'role', 'termination_date', 'type', 'updated_at', 'user_settings', 'user_type_id', 'thumbnail', 'has_login', 'login_type', 'archived_at', '_BATCH_ID_', '_BATCH_LAST_RUN_']

📊 Available roles in the data:
role
Design   

In [ ]:
# [Cell 6] Load Vacation and Leave Data
try:
    df_vacation = pd.read_csv('../data/Namely Vacation and Leave Dataset.csv')
    print(f"✅ Vacation data loaded successfully: {df_vacation.shape}")
    print("\n📊 Column names:")
    print(df_vacation.columns.tolist())
    print("\n🔍 First few rows:")
    print(df_vacation.head())
except Exception as e:
    print(f"❌ Error loading vacation data: {e}")
    df_vacation = None

In [ ]:
# [Cell 7] Load Salesforce Opportunity Data
try:
    df_salesforce = pd.read_csv('../data/Salesforce Opportunity Data.csv')
    print(f"✅ Salesforce data loaded successfully: {df_salesforce.shape}")
    print("\n📊 Column names:")
    print(df_salesforce.columns.tolist())
    print("\n📊 Data types:")
    print(df_salesforce.dtypes)
except Exception as e:
    print(f"❌ Error loading Salesforce data: {e}")
    df_salesforce = None

In [ ]:
# [Cell 8] Load US Working Hours Data
try:
    df_us_hours = pd.read_csv('../data/Working Hours For US.csv')
    print(f"✅ US Working Hours data loaded successfully: {df_us_hours.shape}")
    print("\n📊 Column names:")
    print(df_us_hours.columns.tolist())
    # Convert Month column to datetime
    df_us_hours['Month'] = pd.to_datetime(df_us_hours['Month'])
    print("\n📅 Sample data:")
    print(df_us_hours[['Month', 'Net Working Hours', 'Billable Days']].head(10))
except Exception as e:
    print(f"❌ Error loading US working hours data: {e}")
    df_us_hours = None

In [ ]:
# [Cell 9] Load UAE Working Hours Data
try:
    df_uae_hours = pd.read_csv('../data/UAE Working Hours.csv')
    print(f"✅ UAE Working Hours data loaded successfully: {df_uae_hours.shape}")
    print("\n📊 Column names:")
    print(df_uae_hours.columns.tolist())
    print("\n🔍 First few rows:")
    print(df_uae_hours.head())
except Exception as e:
    print(f"❌ Error loading UAE working hours data: {e}")
    df_uae_hours = None